# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Mentoría 16 - Trabajo práctico 1

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---

## Librerías

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
pd.options.display.float_format = '{:,.2f}'.format

## Dataset

In [ ]:
url = '/home/usuario/muestra_diplodatos_ventas_2023.csv'
ventas = pd.read_csv(url)

---
# Paso 1: Carga y exploración inicial de datos

1.	Cargar los datos.
2.	Explora la estructura de los datos: número de observaciones, variables disponibles, intervalo de tiempo entre observaciones, etc.
3.	Observa las primeras y últimas filas de los datos para tener una idea general de su contenido.
4.	Verifica si hay valores faltantes o atípicos en los datos y propongan cómo manejarlos.


### Variables

Hay un total de 19 variables. Significado tentativo:
*   **ID_VENDEDOR**: identificación del vendedor, anonimizado mediante hasheo. Tipo: objeto.
*   **INSCRIPCION**: desconocemos. Tipo: entero.
*   **AÑO**: año de imputación del registro. Tipo: entero.
*   **MES**: mes de imputación del registro. Tipo: entero.
*   **CATEGORIA**: descripción del Código de Actividad Codiguero NAES Año 2018 y siguientes. Tipo: objeto.
*   **DEPOSITO**: desconocemos. Tipo: entero.
*   **DESCRIPCION_CATEGORIA**: subrubro definido por la DGR (Dato Interno, no declarado por el contribuyente). Tipo: objeto.
*   **TOTAL_VENTAS**: monto total de ventas (base imponible). Tipo: flotante.
*   **PORCENTAJE_COMISION_EMPRESA**: comisión por las ventas en la plataforma (alícuota). Tipo: flotante.
*   **COMISION_EMPRESA**: comisión por las ventas en la plataforma (valor). Tipo: flotante.
*   **TRATAMIENTO_FISCAL**: indica qué tratamiento fiscal se le da a la operación, especialmente respecto a la alícuota a cobrarle. Tipo: objeto.
*   **DESC_TRATAMIENTO_FISCAL**: categoriza los valores de `TRATAMIENTO_FISCAL`. Tipo: objeto.
*   **TRATAMIENTO_DIFERNCIAL**: indica el artículo de alguna reglamentación aplicado para dar tratamiento fiscal especial. Tipo: objeto.
*   **CM04**: puede marcar las operaciones bajo Convenio Multilateral (CM) o indicar que se trata de un vendedor inscripto en el CM. Tipo: objeto.
*   **CATEGORIA (Ajustado)**: variable `CATEGORIA` limpia (se redujo el número de valores posibles). Tipo: objeto.
*   **SUB-CATEGORIA**: variable `DESCRIPCION_CATEGORIA` limpia (se redujo el número de valores posibles). Tipo: objeto.
*   **NOMBRE**: desconocemos. Se trata de otra variable anonimizada mediante hasheo, pero es un hash diferente al de `ID_VENDEDOR`. Tipo: objeto.
*   **OMEGA**: marca para contribuyente de interés fiscal especial. Tipo: entero.
*   **MODELO**: identificación de vendedores modelo según la empresa proveedora de los datos. Tipo: entero.

In [ ]:
lista_columnas = list(ventas.columns)
print(len(lista_columnas), 'variables:')
lista_columnas

In [ ]:
ventas.dtypes

In [ ]:
display(ventas[:10])

### Estadística descriptiva

De las 19 variables, 9 son numéricas.
*   **INSCRIPCION**: al no conocer a qué hace referencia, no podemos interpretar la estadística descriptiva.
*   **DEPOSITO**: al no conocer a qué hace referencia, no podemos interpretar la estadística descriptiva.
*   **AÑO**: desde 2019 hasta 2022.
*   **MES**: los 12 meses del año.
*   **PORCENTAJE_COMISION_EMPRESA**: desde el 0 hasta el 18%.
*   **OMEGA**: todos tienen valor igual a 1, por lo que esta variable no aporta nada de información.
*   **MODELO**: se tienen valores 0 y 1 para identificar vendedores no modelo de vendedores modelo, respectivamente.
*   **TOTAL_VENTAS**: la dispersión de datos es enorme: la desviación estándar es un orden mayor que la media. Inlcuso hay valores negativos: hay que determinar si están mal imputados o si el hecho de ser negativos tiene un significado. Analizando los z-Scores vemos que el mínimo está a 5 mientras que el máximo está a 101 desviaciones estándares respecto a la media.
*   **COMISION_EMPRESA**: observaciones análogas al punto anterior. Analizando los z-Scores vemos que el mínimo está a 3 mientras que el máximo está a 131 desviaciones estándares respecto a la media.

In [ ]:
ventas.describe().T

In [ ]:
# TOTAL_VENTAS
zScore_min = (ventas['TOTAL_VENTAS'].min() - ventas['TOTAL_VENTAS'].mean()) / ventas['TOTAL_VENTAS'].std()
zScore_max = (ventas['TOTAL_VENTAS'].max() - ventas['TOTAL_VENTAS'].mean()) / ventas['TOTAL_VENTAS'].std()

print('z-Score de los extremos de la variable "TOTAL_VENTAS":')
print(f'\t Mínimo: {zScore_min:.0f}')
print(f'\t Mínimo: {zScore_max:.0f}')

# COMISION_EMPRESA
zScore_min = (ventas['COMISION_EMPRESA'].min() - ventas['COMISION_EMPRESA'].mean()) / ventas['COMISION_EMPRESA'].std()
zScore_max = (ventas['COMISION_EMPRESA'].max() - ventas['COMISION_EMPRESA'].mean()) / ventas['COMISION_EMPRESA'].std()

print('z-Score de los extremos de la variable "COMISION_EMPRESA":')
print(f'\t Mínimo: {zScore_min:.0f}')
print(f'\t Mínimo: {zScore_max:.0f}')

### Datos faltantes

Vemos que tenemos un total de 431.506 registros. Entre las variables disponibles tenemos 5 con valores faltantes:
* **CM04:** sólo hay 42 registros (menos del 0.01% del total de registros).
* **TRATAMIENTO_DIFERNCIAL:** tiene 83.058 reigstros (cerca del 20% del total de registros).
* **DESC_TRATAMIENTO_FISCAL:** tiene 117.841 registros (cerca del 30% del total de registros).
* **TRATAMIENTO_FISCAL:** tiene 403.538 registros (más del 90% del total de registros).
* **NOMBRE:** tiene 430.857 registros (casi el total de registros).

In [ ]:
msno.bar(ventas,figsize=(12, 3), sort="ascending",fontsize=12, color='gray')
plt.show()

### Cardinalidad: valores únicos

`ID_VENDEDOR` tiene 3209 valores (vendedores) diferentes. El vendedor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.85%.

In [ ]:
m = 0
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`INSCRIPCION` tiene 3345 valores diferentes. El valor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.72%. Hay 136 valores más que en `ID_VENDEDOR`, pero porcentajes similares: puede existir una relación entre ambas, la cual no necesariamente es biyectiva.

In [ ]:
m = 1
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`AÑO` tiene 4 valores diferentes: los 4 años que ya dijimos. El 2021 contribuye en un 29.69% de los registros, mientras que el 2022 con 15.02%.

In [ ]:
m = 2
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`MES` tiene 12 valores diferentes: los 12 meses del año. Junio es el que más contribuye con el 9.56% de los registros. Los 10 meses con mayores contribuciones suman un total del 85.78% de registros.

In [ ]:
m = 3
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CATEGORIA` tiene 5 valores diferentes, pero parece que en realidad son todas la misma categoría, con diferentes formatos de escritura.

In [ ]:
m = 4
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DEPOSITO` tiene 241 valores diferentes. El valor mayoritario contribuye en un 5.21% de los registros y los 10 primeros en conjunto contribuyen al 26.64%.

In [ ]:
m = 5
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DESCRIPCION_CATEGORIA` tiene 245 valores diferentes. El valor mayoritario contribuye en un 5.21% de los registros y los 10 primeros en conjunto contribuyen al 26.64%. La cantidad de valores es similar al de la variable `DEPOSITO` y, además, las contribuciones porcentuales son simialres. Puede existir una relación entre ambas variables.

In [ ]:
m = 6
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TOTAL_VENTAS` tiene 248974 valores diferentes. El valor mayoritario contribuye en un 41.96% de los registros, indicando ventas por un valor de $0. Esto es sospechoso: ¿es un indicio para vendedores que hacen fuga de ventas?. Por otro lado, los 10 primeros en conjunto contribuyen al 42.06%.

In [ ]:
m = 7
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`PORCENTAJE_COMISION_EMPRESA` tiene 89 valores diferentes. El valor mayoritario (4.75%) contribuye en un 19.41% de los registros y los 10 primeros en conjunto contribuyen al 73.27%.

In [ ]:
m = 8
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`COMISION_EMPRESA` tiene 244284 valores diferentes. El valor mayoritario contribuye en un 42.52% de los registros y los 10 primeros en conjunto contribuyen al 42.56%. Nuevamente el valor mayoritario es de $0 como en `TOTAL_VENTAS`. Tiene sentido que estas variables estén relacionadas.

In [ ]:
m = 9
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TRATAMIENTO_FISCAL` tiene 17 valores diferentes: algunos numéricos y otros categóricos. El valor mayoritario contribuye en un 45.50% de los registros y los 10 primeros en conjunto contribuyen al 99.16%. Hay que unificar cosas como 0 y 0.0. Además ¿existe alguna relación entre ser 1 y ser especial 1? (o similares)

In [ ]:
m = 10
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`DESC_TRATAMIENTO_FISCAL` tiene 4 valores diferentes. El valor mayoritario contribuye en un 94.36% de los registros. ¿Qué relación hay entre esta variable y `TRATAMIENTO_FISCAL`?

In [ ]:
m = 11
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`TRATAMIENTO_DIFERNCIAL` tiene 11 valores diferentes. El valor mayoritario (Art. 21) contribuye en un 34.45% de los registros y los 10 primeros en conjunto contribuyen al 99.52%. ¿Cómo tratar a los casos "Art. 19", "Art. 20" y "Art. 19 y 20"?

In [ ]:
m = 12
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CM04` tiene un único valor: "Si". Esta variable tenía casi el 100% de registros vacíos: ¿estar vacío implica un "No"?

In [ ]:
m = 13
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`CATEGORIA (Ajustado)` tiene un único valor: en efecto la variable `CATEGORIA` presentaba un único valor, escrito de formas diferentes.

In [ ]:
m = 14
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`SUB-CATEGORIA` tiene 21 valores diferentes: las 245 opciones de `DESCRIPCION_CATEGORIA` fueron reducidas a esta cantidad. El valor mayoritario contribuye en un 32.51% de los registros y los 10 primeros en conjunto contribuyen al 87.78%. Si existía una relación con `DEPOSITO` ¿se sigue apreciando en esta nueva variable?

In [ ]:
m = 15
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`NOMBRE` tiene 3199 valores diferentes. El valor mayoritario contribuye en un 0.39% de los registros y los 10 primeros en conjunto contribuyen al 2.85%. Si bien los porcentajes son similares a aquellos encontrados para `ID_VENDEDOR`, los hashs son diferentes y, además, hay 10 nombres menos.

In [ ]:
m = 16
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`OMEGA` tiene un único valor como habíamos dicho, por lo que no aporta nada de información relevante.

In [ ]:
m = 17
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

`MODELO` tiene 2 valores diferentes como habíamos dicho. Sólo el 0.32% de los registros es considerado **vendedor modelo** por parte del cliente.

In [ ]:
m = 18
a = 100*ventas[ventas.columns[m]].value_counts(normalize=True).iloc[:10]
print(a)
print(f'\nContribución porcentual de los 10 casos mayoritarios: {np.sum(a):.2f} %')
print(f'Cardinalidad: {ventas[ventas.columns[m]].nunique()}')

### Variables que no contribuyen

En base a lo visto hasta aquí, se podría prescindir de las variables `CATEGORIA` y `DESCRIPCION_CATEGORIA`, las cuales fueron superadas por `CATEGORIA (Ajustado)` y `SUB-CATEGORIA`, respectivamente. Sin embargo, como `CATEGORIA (Ajustado)` presenta un único valor posible, también es prescindible. Lo mismo ocurre con `OMEGA`.

In [ ]:
ventas_clean = ventas.drop(['CATEGORIA', 'DESCRIPCION_CATEGORIA', 'OMEGA', 'CATEGORIA (Ajustado)'], axis=1).copy()
display(ventas_clean[:3])

### Construcción de una nueva variable: Fecha

Consideramos que para el análisis temporal habrá momentos en que necesitamos tener año y mes por separado, pero en otras ocasiones necesitamos toda la información al mismo tiempo. Por lo tanto, creamos la variable fecha:
    $$FECHA = AÑO + MES$$

In [ ]:
ventas_clean['FECHA'] = pd.to_datetime(ventas_clean['MES'].astype(str) + '-' + ventas_clean['AÑO'].astype(str), format='%m-%Y').dt.to_period('M')
display(ventas_clean[:3])

In [ ]:
ventas_clean['FECHA'].dtypes

### Algunas pruebas

#### Significado de los registros

Para saber qué representa cada registro (ventas mensuales, vendedores por mes, etcétera). Vemos que hay más de una observación para cada fecha por vendedor. Probablemente cada registro represente operaciones de ventas individuales.

In [ ]:
prueba1 = ventas_clean[['ID_VENDEDOR', 'FECHA']].copy()

In [ ]:
for k in range(len(prueba1['ID_VENDEDOR'])):
    p = prueba1[prueba1['ID_VENDEDOR']==prueba1['ID_VENDEDOR'][k]]
    p = np.sum(p.duplicated(subset='FECHA'))
    if p != 0:
        print('Existen vendedores con más de un registro en la misma "FECHA".')
        break

In [ ]:
# Vamos a eliminar duplicados por ID_VENDEDOR y FECHA
prueba1 = ventas_clean.drop_duplicates(subset=['ID_VENDEDOR', 'FECHA']).copy()
print(len(prueba1))
print(len(ventas_clean))

#### Relación entre `ID_VENDEDOR` y `NOMBRE`

Queremos saber si existe una corrspondencia entre estas 2 variables. Ya vimos que había 10 nombres menos. Ahora vemos que hay más de un registro que asocia un dado `ID_VENDEDOR` con cierto `NOMBRE`. Falta saber qué representa `NOMBRE`para saber si se puede descartar o no.

In [ ]:
prueba2 = ventas_clean[['ID_VENDEDOR', 'NOMBRE']].copy()

In [ ]:
for k in range(len(prueba2['ID_VENDEDOR'])):
    p = prueba2[prueba2['ID_VENDEDOR']==prueba2['ID_VENDEDOR'][k]]
    p = np.sum(p.duplicated(subset='NOMBRE'))
    if p != 0:
        print('Existen vendedores con más de un registro con el mismo "NOMBRE".')
        break

#### Unificacón de categorías de `TRATAMIENTO_FISCAL`

Parece que la variable asume valores enteros, floats y strings. Sin embargo, se corrobora que los que parecen enteros, en realidad son strings. Vamos a forzar los floats hacia strings.

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'].unique()

In [ ]:
ventas_clean['TRATAMIENTO_FISCAL'] = ventas_clean['TRATAMIENTO_FISCAL'].replace({0.0: '0', 3.0: '3', 2.0: '2', 1.0: '1'})
ventas_clean['TRATAMIENTO_FISCAL'].value_counts()

#### Relación entre `TRATAMIENTO_FISCAL` y `DESC_TRATAMIENTO_FISCAL`

Queremos chequear para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `DESC_TRATAMIENTO_FISCAL`. Al analizar la tabla de contingencia, vemos que los valores de `DESC_TRATAMIENTO_FISCAL` sólo aparecen cuando `TRATAMIENTO_FISCAL` asume valores 0, 1, 2 ó 3. A su vez, hay correspondencia entre 0 y Normal, 1 y Exento/Desgravado, 2 y Minorista, 3 y Otro Tratamiento Fiscal. ¿Se puede unificar todo como `TRATAMIENTO_FISCAL`, eliminando entonces `DESC_TRATAMIENTO_FISCAL`? ¿Tiene sentido que falte la descripción asociada al tratamiento?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3 = ventas_clean.copy()
prueba3['TRATAMIENTO_FISCAL'] = prueba3['TRATAMIENTO_FISCAL'].fillna('')
prueba3['DESC_TRATAMIENTO_FISCAL'] = prueba3['DESC_TRATAMIENTO_FISCAL'].fillna('')

In [ ]:
pd.crosstab(prueba3['TRATAMIENTO_FISCAL'], prueba3['DESC_TRATAMIENTO_FISCAL'], dropna=False)

#### Relación entre `TRATAMIENTO_FISCAL` y `TRATAMIENTO_DIFERENCIAL`

Ahora veamos para qué valores de `TRATAMIENTO_FISCAL` aparecen valores de `TRATAMIENTO_DIFERNCIAL`. Al analizar la tabla de contingencia vemos que no hay un patrón claro. ¿Podríamos tomar a un dato faltante como "sin tratamiento" o algo por el estilo?

In [ ]:
# Este paso lo agrego porque sino el crosstab siguiente no me muestra los NaN
prueba3['TRATAMIENTO_DIFERNCIAL'] = prueba3['TRATAMIENTO_DIFERNCIAL'].fillna('')

In [ ]:
pd.crosstab(prueba3['TRATAMIENTO_FISCAL'], prueba3['TRATAMIENTO_DIFERNCIAL'], dropna=False)

#### Relación descriptiva entre `TOTAL_VENTAS`, `PORCENTAJE_COMISION_EMPRESA` y `COMISION_EMPRESA`

Como ya vimos, existen valores negativos en `TOTAL_VENTAS` y `COMISION_EMPRESA`. ¿Tiene algún significado el signo negativo o es un error? También hay muchos registros nulos: ¿tiene algún significado especial o es un indicio de fuga? 

Destacamos nuevamente que hay una dispersión muy grande, presentándose la mayor asimetría hacia la derecha.

Vemos que poco más del 42% de los registros poseen un valor negativo o nulo en alguna de estas 2 variables.

Analizando los porcentajes de comisión, ¿tiene sentido que haya porcentajes nulos?

Hay que decidir:
* Qué hacemos con valores negativos.
* Qué hacemos con valores nulos.
* Si truncamos valores extremos y, en caso afirmativo, la manera/los límites.

In [ ]:
ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']].describe()

In [ ]:
# TOTAL_VENTAS
ans = len(ventas_clean)
neg = len(ventas_clean[ventas_clean["TOTAL_VENTAS"] < 0])
nul = len(ventas_clean[ventas_clean["TOTAL_VENTAS"] == 0])
print(f'Valores negativos:')
print(f'\t{neg} de {ans}.')
print(f'\t{100*neg/ans:.2f}%')

print(f'Valores nulos:')
print(f'\t{nul} de {ans}.')
print(f'\t{100*nul/ans:.2f}%')

print(f'Valores negativos más nulos:')
print(f'\t{nul+neg} de {ans}.')
print(f'\t{100*(nul+neg)/ans:.2f}%')

In [ ]:
# COMISION_EMPRESA
ans = len(ventas_clean)
neg = len(ventas_clean[ventas_clean["COMISION_EMPRESA"] < 0])
nul = len(ventas_clean[ventas_clean["COMISION_EMPRESA"] == 0])
print(f'Valores negativos:')
print(f'\t{neg} de {ans}.')
print(f'\t{100*neg/ans:.2f}%')

print(f'Valores nulos:')
print(f'\t{nul} de {ans}.')
print(f'\t{100*nul/ans:.2f}%')

print(f'Valores negativos más nulos:')
print(f'\t{nul+neg} de {ans}.')
print(f'\t{100*(nul+neg)/ans:.2f}%')

In [ ]:
# Crear la figura y los subplots
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

# Boxplot para 'TOTAL_VENTAS'
axs[0].boxplot(ventas_clean['TOTAL_VENTAS'].dropna())
axs[0].set_title('Boxplot de TOTAL_VENTAS')

# Boxplot para 'COMISION_EMPRESA'
axs[1].boxplot(ventas_clean['COMISION_EMPRESA'].dropna())
axs[1].set_title('Boxplot de COMISION_EMPRESA')

# Boxplot para 'PORCENTAJE_COMISION_EMPRESA'
axs[2].boxplot(ventas_clean['PORCENTAJE_COMISION_EMPRESA'].dropna())
axs[2].set_title('Boxplot de PORCENTAJE_COMISION_EMPRESA')

# Ajustar los espacios entre subplots
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
prueba4 = ventas_clean[['TOTAL_VENTAS','PORCENTAJE_COMISION_EMPRESA','COMISION_EMPRESA']]

In [ ]:
sns.pairplot(data=prueba4, diag_kind='kde', plot_kws = {'alpha': 0.3, 's': 10})
plt.show()

#### Relación numérica entre `TOTAL_VENTAS`, `PORCENTAJE_COMISION_EMPRESA` y `COMISION_EMPRESA`

Queremos saber si estas 3 variables están relacionadas entre sí mediante alguna fórmula. 

In [ ]:
# Probemos con verificar la siguiente fórmula:
ventas_clean['CALCULADA'] = ventas_clean['TOTAL_VENTAS'] * ventas_clean['PORCENTAJE_COMISION_EMPRESA']

In [ ]:
# Cuándo hay coincidencia exacta?
sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA'] == 0))

In [ ]:
# Es decir, sólo en 191 mil casos la fórmula se cumple exactamente.

In [ ]:
# Probemos con algún margen de error

In [ ]:
sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']) <= 0.01)

In [ ]:
for x in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
  print('Para', x, 'de diferencia, tenemos', sum(abs(ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']) <= x), 'igualdades')

In [ ]:
# Para 261 mil casos (61%) la fórmula se verifica, con precisión menor a 1 entero.

In [ ]:
# Grafiquemos la distribución de la diferencia

diferencia = ventas_clean['COMISION_EMPRESA'] - ventas_clean['CALCULADA']
plt.boxplot(diferencia.dropna())
plt.xlabel('Diferencia')
plt.ylabel('Valor')
plt.title('Box Plot de la diferencia entre COMISION_EMPRESA y CALCULADA')
plt.show()

In [ ]:
# Surge otra pregunta: habrá algun caso que la diferencia es positiva?
diferencia.describe()

In [ ]:
plt.scatter(ventas_clean['COMISION_EMPRESA'], ventas_clean['CALCULADA'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('CALCULADA')
plt.title('COMISION_EMPRESA vs CALCULADA')
plt.show()

In [ ]:
# Poniendo foco en los casos en que COMISION_EMPRESA es mayor que CALCULADA:
plt.scatter(ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']]['COMISION_EMPRESA'], ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']]['CALCULADA'])
plt.xlabel('COMISION_EMPRESA')
plt.ylabel('CALCULADA')
plt.title('COMISION_EMPRESA vs CALCULADA')
plt.show()

In [ ]:
len(ventas_clean[ventas_clean['COMISION_EMPRESA'] > ventas_clean['CALCULADA']])

In [ ]:
# Lo que figura en la variable COMISION_EMPRESA siempre es menor o igual que lo que surgiría de multiplicar TOTAL_VENTAS y PORCENTAJE_COMISION_EMPRESA

## Paso 2: Análisis estadístico descriptivo

1.	Calcula estadísticas descriptivas básicas de la serie temporal, como la media, mediana, desviación estándar, mínimo y máximo.


In [ ]:
ventas['TOTAL_VENTAS'].describe()

2.	Analiza la tendencia central y la dispersión de los datos. ¿Existen valores atípicos o extremos? ¿Cómo podrían afectar el análisis posterior?


In [ ]:
#PROPONER ALGUN PERCENTIL. Y PONER EL NRO ABSOLUTO EN CANTIDAD DE REGISSTRO

In [ ]:
plt.boxplot(ventas_clean['TOTAL_VENTAS'].dropna())
plt.title('TOTAL_VENTAS')
plt.show()

In [ ]:
# Los valores negativos y los 0 sesgan hacia abajo la media
# Los valores extremos positivos sesgan hacia arriba

In [ ]:
# Negativos
len(ventas_clean[ventas_clean['TOTAL_VENTAS'] < 0])

In [ ]:
# Ceros
(ventas_clean['TOTAL_VENTAS'] == 0).sum()

In [ ]:
# Extremos positivos
percentil_95 = np.percentile(ventas_clean['TOTAL_VENTAS'], 95)
len(ventas_clean[ventas_clean['TOTAL_VENTAS'] >= percentil_95])

In [ ]:
# Al eliminar los extremos positivos y negativos
plt.boxplot(ventas_clean[(ventas_clean['TOTAL_VENTAS'] > 0) & (ventas_clean['TOTAL_VENTAS'] < percentil_95)]['TOTAL_VENTAS'])
plt.title('TOTAL_VENTAS')
plt.show()

In [ ]:
#Agregar histograma

3.	Grafica la serie temporal en un gráfico de líneas para visualizar la evolución de los datos a lo largo del tiempo.

In [ ]:
prueba6 = ventas_clean[['FECHA','TOTAL_VENTAS']].copy()

In [ ]:
prueba6['FECHA'].dtype

In [ ]:
prueba6['TOTAL_VENTAS'].dtype

In [ ]:
prueba6_agregado = prueba6.groupby('FECHA')['TOTAL_VENTAS'].sum().reset_index()

plt.figure(figsize=(16, 3))

# Graficar el gráfico de líneas
plt.plot(prueba6_agregado['FECHA'].astype(str), prueba6_agregado['TOTAL_VENTAS'])

# Personalizar el gráfico
plt.title('Total de ventas')
plt.xlabel('Fecha')
plt.ylabel('Ventas')

# Rotar las etiquetas del eje X para mayor legibilidad (opcional)
plt.xticks(rotation=45)

# Mostrar el gráfico
plt.show()

In [ ]:
len(prueba6_agregado)

In [ ]:
# GRAFICAR POR AÑO. LA TENDENCIA DE MESES. PARA VER ESTACIONALIDAD.

In [ ]:
# Se observa el efecto de la inflación y pareciera haber picos en noviembre/diciembre

4.	Calcula y grafica la función de autocorrelación para identificar posibles patrones de autocorrelación en los datos. (Es un término estadístico que se utiliza para describir la presencia o ausencia de correlación en los datos de las series temporales, indicando, si las observaciones pasadas influyen en las actuales.)

In [ ]:
# Calcular la función de autocorrelación
acf = plot_acf(prueba6_agregado['TOTAL_VENTAS'], lags=12)

# Graficar la función de autocorrelación
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.show()

In [ ]:
# Hay correlación estadísticamente significativa para el lag 1, 2 y 3.
# Lo que implica que observaciones pasadas influyen en las observaciones actuales de la serie temporal.
# De manera positiva